<a href="https://colab.research.google.com/github/pedrodeparis/Hyperparameter-optimization/blob/main/Tunning_GridSearch_RandomizedSearch_LightGBM_Churn_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_excel("churn_data.xlsx")
df.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,45,No,One year,No,Bank transfer,42.30,1840.75,No
4,9237-HQITU,2,Yes,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors = "coerce")


In [7]:
df = df.dropna()

In [9]:
cat_vars = ["PhoneService", "Contract", "PaperlessBilling", "PaymentMethod"]
num_vars = ["tenure", "MonthlyCharges", "TotalCharges"]
modeling_vars = cat_vars + num_vars

In [11]:
X = df.filter(modeling_vars).copy()
y = df["Churn"].copy()
y = pd.DataFrame(y)

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(y.Churn)
y.Churn = le.transform(y.Churn)

In [13]:
X = pd.get_dummies(X)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier()

parameters = {
    "max_depth": [1,2,3,4,5,6,7,8,9,10]
}

grid_search = GridSearchCV(arvore, parameters, scoring="roc_auc", cv= 5, n_jobs=-1)

In [18]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             scoring='roc_auc')

In [19]:
results = pd.DataFrame(grid_search.cv_results_)

In [20]:
results.sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,0.018364,0.000660,0.013075,0.001914,5,{'max_depth': 5},0.844178,0.826520,0.847247,0.822491,0.822203,0.832528,0.010916,1
3,0.017062,0.001160,0.013921,0.003564,4,{'max_depth': 4},0.843354,0.821277,0.846777,0.824119,0.826509,0.832407,0.010523,2
5,0.023423,0.006048,0.015414,0.004412,6,{'max_depth': 6},0.832667,0.815610,0.844224,0.813776,0.819832,0.825222,0.011567,3
2,0.014603,0.000265,0.012202,0.000619,3,{'max_depth': 3},0.831087,0.804024,0.838196,0.803929,0.811126,0.817672,0.014277,4
6,0.021317,0.000209,0.011958,0.000758,7,{'max_depth': 7},0.820024,0.796122,0.835525,0.800924,0.818620,0.814243,0.014221,5
7,0.024463,0.002130,0.011566,0.000606,8,{'max_depth': 8},0.803678,0.790326,0.801505,0.776124,0.799437,0.794214,0.010123,6
1,0.012658,0.000766,0.013463,0.001294,2,{'max_depth': 2},0.781847,0.781078,0.789014,0.781129,0.777095,0.782033,0.003870,7
8,0.026595,0.003456,0.012294,0.001148,9,{'max_depth': 9},0.785006,0.776108,0.795402,0.759201,0.786314,0.780406,0.012240,8
9,0.025541,0.001528,0.010683,0.001785,10,{'max_depth': 10},0.772132,0.755266,0.781783,0.714043,0.753264,0.755298,0.023199,9
0,0.013050,0.003395,0.015355,0.003878,1,{'max_depth': 1},0.723092,0.718568,0.730212,0.728109,0.733009,0.726598,0.005162,10


In [26]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

In [27]:
parameters = {"max_depth": [1,2,3,4,5,6,7,8,9,10],
              "n_estimators": [100,200,500]}

grid_search = GridSearchCV(rf, parameters, scoring="roc_auc", cv=5, n_jobs = -1)

In [28]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [100, 200, 500]},
             scoring='roc_auc')

In [29]:
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values(by="rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
19,1.662381,0.293955,0.079874,0.024299,7,200,"{'max_depth': 7, 'n_estimators': 200}",0.846626,0.830656,0.858728,0.837762,0.840134,0.842781,0.009469,1
20,2.970737,0.024484,0.147587,0.002137,7,500,"{'max_depth': 7, 'n_estimators': 500}",0.847279,0.829938,0.858981,0.836598,0.840314,0.842622,0.009918,2
17,2.775274,0.031159,0.141244,0.004597,6,500,"{'max_depth': 6, 'n_estimators': 500}",0.847614,0.828856,0.857032,0.838007,0.839795,0.842261,0.009494,3
18,0.604163,0.018831,0.040514,0.002100,7,100,"{'max_depth': 7, 'n_estimators': 100}",0.848901,0.827063,0.859455,0.835748,0.837659,0.841765,0.011252,4
16,1.559338,0.301092,0.101169,0.018285,6,200,"{'max_depth': 6, 'n_estimators': 200}",0.847931,0.829195,0.856240,0.837370,0.837324,0.841612,0.009429,5


In [30]:
grid_search.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [31]:
from lightgbm import LGBMClassifier


lgbm = LGBMClassifier(n_jobs=-1)

In [33]:
from sklearn.model_selection import RandomizedSearchCV

parameters = {"learning_rate":[0.001, 0.01],
              "num_leaves": [2, 128],
              "min_child_samples": [1, 100],
              "subsample": [0.05,1],
              "colsample_bytree": [0.1, 1.0]
              }

random_search = RandomizedSearchCV(lgbm, parameters, scoring="roc_auc", cv=5, n_iter=5, n_jobs=-1)

random_search.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1495, number of negative: 4129
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000371 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 605
[LightGBM] [Info] Number of data points in the train set: 5624, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265825 -> initscore=-1.015909
[LightGBM] [Info] Start training from score -1.015909


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


RandomizedSearchCV(cv=5, estimator=LGBMClassifier(n_jobs=-1), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.1, 1.0],
                                        'learning_rate': [0.001, 0.01],
                                        'min_child_samples': [1, 100],
                                        'num_leaves': [2, 128],
                                        'subsample': [0.05, 1]},
                   scoring='roc_auc')

In [34]:
results = pd.DataFrame(random_search.cv_results_)
results.sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_num_leaves,param_min_child_samples,param_learning_rate,param_colsample_bytree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.321675,0.257077,0.026637,0.006552,1,2,1,0.001,0.1,"{'subsample': 1, 'num_leaves': 2, 'min_child_s...",0.824459,0.811559,0.831513,0.825352,0.816238,0.821824,0.007069,1
1,0.312414,0.253225,0.036373,0.011631,0.05,2,1,0.001,0.1,"{'subsample': 0.05, 'num_leaves': 2, 'min_chil...",0.824459,0.811559,0.831513,0.825352,0.816238,0.821824,0.007069,1
4,0.303220,0.193177,0.029944,0.011455,0.05,2,100,0.001,0.1,"{'subsample': 0.05, 'num_leaves': 2, 'min_chil...",0.824459,0.811559,0.831513,0.825352,0.816238,0.821824,0.007069,1
2,0.365115,0.360344,0.055825,0.023838,1,2,1,0.01,1.0,"{'subsample': 1, 'num_leaves': 2, 'min_child_s...",0.826221,0.803987,0.813861,0.800694,0.807690,0.810491,0.008999,4
3,0.572719,0.628858,0.046120,0.016951,1,2,100,0.001,1.0,"{'subsample': 1, 'num_leaves': 2, 'min_child_s...",0.723092,0.718568,0.730212,0.728109,0.733009,0.726598,0.005162,5
